In [5]:
import zipfile
import requests
import io
import pandas as pd
# URL of MovieLens 100k zip
url = "https://files.grouplens.org/datasets/movielens/ml-100k.zip"

# Download and unzip in memory
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('ml-100k/u.data') as f1:
        ratings = pd.read_csv(f1, sep='\t', names=['user_id', 'movie_id', 'rating'], encoding='ISO-8859-1')
    with z.open('ml-100k/u.item') as f2:
        movies = pd.read_csv(f2, sep='|', names=['movie_id', 'title'], usecols=[0, 1], encoding='ISO-8859-1')

ratings=pd.merge(ratings,movies)

In [7]:
ratings.head(10)

,user_id,movie_id,rating,title
0,242,3,881250949,Four Rooms (1995)
1,302,3,891717742,Four Rooms (1995)
2,377,1,878887116,Toy Story (1995)
3,51,2,880606923,GoldenEye (1995)
4,346,1,886397596,Toy Story (1995)
5,474,4,884182806,Get Shorty (1995)
6,265,2,881171488,GoldenEye (1995)
7,465,5,891628467,Copycat (1995)
8,451,3,886324817,Four Rooms (1995)
9,86,3,883603013,Four Rooms (1995)


In [15]:
import numpy as np

movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

/tmp/ipykernel_4085/1254925699.py:3: FutureWarning: The provided callable <function mean at 0x73fd060659e0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})


,size
movie_id,
1,0.000000
2,0.187429
3,0.749537
4,1.000000
5,0.537735


In [ ]:

with z.open('ml-100k/u.item') as f:
    movie_dict = {}
    for line in f:
        line = line.decode('ISO-8859-1')
        fields = line.rstrip('\n').split('|')
        movie_id = int(fields[0])
        name = fields[1]
        genre = np.array(list(map(int, fields[5:25])))
        # Use .get to avoid KeyError if movie_id is missing
        size = movieNormalizedNumRatings['size'].get(movie_id, np.nan)
        mean = movieProperties['rating']['mean'].get(movie_id, np.nan)
        movie_dict[movie_id] = (name, genre, size, mean)
# To access info for movie_id 1:
movie_dict.get(1)

ValueError: Attempt to use ZIP archive that was already closed

In [9]:
corrMatrix=movieRatings.corr(method='pearson',min_periods=100)
corrMatrix

title,Copycat (1995),Four Rooms (1995),Get Shorty (1995),GoldenEye (1995),Toy Story (1995)
title,,,,,
Copycat (1995),1.000000,0.250287,0.255625,0.184440,0.183435
Four Rooms (1995),0.250287,1.000000,0.345131,0.236943,0.223543
Get Shorty (1995),0.255625,0.345131,1.000000,0.258819,0.219815
GoldenEye (1995),0.184440,0.236943,0.258819,1.000000,0.192220
Toy Story (1995),0.183435,0.223543,0.219815,0.192220,1.000000


In [10]:
user_rating=movieRatings.loc[1].dropna()
user_rating

title
Copycat (1995)       8.823133e+08
Four Rooms (1995)    8.825176e+08
Get Shorty (1995)    8.828745e+08
GoldenEye (1995)     8.857561e+08
Toy Story (1995)     8.818722e+08
Name: 1, dtype: float64

In [11]:
simcondidates = pd.Series(dtype='float64')

for i in range(len(user_rating)):
    movie = user_rating.index[i]
    rating = user_rating.iloc[i]
    
    if movie in corrMatrix.columns:
        print(f"Adding similarity for: {movie} (rating: {rating})")
        similarity = corrMatrix[movie].dropna()
        similarity = similarity.map(lambda x: x * rating)
        simcondidates = simcondidates.add(similarity, fill_value=0)
    else:
        print(f"Skipping {movie}, not in corrMatrix")

print("Sorting...")
simcondidates.sort_values(inplace=True, ascending=False)

# Filter out already-rated movies
recommendations = simcondidates.drop(labels=user_rating.index, errors='ignore')

print("Top 10 Recommendations:")
print(recommendations.head(10))


Adding similarity for: Copycat (1995) (rating: 882313308.4033613)
Adding similarity for: Four Rooms (1995) (rating: 882517621.46875)
Adding similarity for: Get Shorty (1995) (rating: 882874523.039604)
Adding similarity for: GoldenEye (1995) (rating: 885756100.6296296)
Adding similarity for: Toy Story (1995) (rating: 881872165.875)
Sorting...
Top 10 Recommendations:
Series([], dtype: float64)


adding similarity for  Copycat (1995) ...
adding similarity for  Four Rooms (1995) ...
adding similarity for  Get Shorty (1995) ...
adding similarity for  GoldenEye (1995) ...
adding similarity for  Toy Story (1995) ...
sorting...


/tmp/ipykernel_8705/501517445.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  simaliraty= simaliraty.map(lambda x : x *user_rating[i])


title
Copycat (1995)       NaN
Four Rooms (1995)    NaN
Get Shorty (1995)    NaN
GoldenEye (1995)     NaN
Toy Story (1995)     NaN
dtype: object